In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [23]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).values
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [24]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf

In [11]:
from tensorflow import keras

In [15]:
# Importing libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [16]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [18]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [25]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [26]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [27]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [29]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 16s 31ms/step - loss: 1.3175 - accuracy: 0.4683 - val_loss: 1.1413 - val_accuracy: 0.5273
Epoch 2/30
460/460 [==============================] - 14s 30ms/step - loss: 0.7967 - accuracy: 0.6382 - val_loss: 0.6778 - val_accuracy: 0.7044
Epoch 3/30
460/460 [==============================] - 14s 30ms/step - loss: 0.6046 - accuracy: 0.7539 - val_loss: 0.6884 - val_accuracy: 0.7435
Epoch 4/30
460/460 [==============================] - 14s 30ms/step - loss: 0.5133 - accuracy: 0.8099 - val_loss: 0.5713 - val_accuracy: 0.8130
Epoch 5/30
460/460 [==============================] - 14s 30ms/step - loss: 0.4284 - accuracy: 0.8703 - val_loss: 0.5202 - val_accuracy: 0.8351
Epoch 6/30
460/460 [==============================] - 14s 30ms/step - loss: 0.3339 - accuracy: 0.8998 - val_loss: 0.4960 - val_accuracy: 0.8476
Epoch 7/30
460/460 [==============================] - 14s 30ms/step - loss: 0.2639 - accuracy: 0.9154 - val_loss: 0.4132 - val_accuracy:

In [30]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      373       116        0                   1   
STANDING                 0       84       448        0                   0   
WALKING                  0        2         1      453                  13   
WALKING_DOWNSTAIRS       0        0         0        3                 413   
WALKING_UPSTAIRS         1        4         1        5                   3   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            1  
STANDING                           0  
WALKING                           27  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 457  


In [31]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 1s 8ms/step - loss: 0.3695 - accuracy: 0.9006


In [32]:
score

[0.36945927143096924, 0.9005768299102783]

- With a simple 2 layer architecture we got 90.06% accuracy and a loss of 0.36
- We need to further imporve the performace with Hyperparameter tuning

### Hyper Parameter Tuning